# Python Scheduling Application 

In [1]:
import os
from msilib import Binary

from pyomo.environ import *
from pyomo.opt import SolverFactory

import pandas as pd

In [2]:
os.environ["NEOS_EMAIL"] = "manyu1994@hotmail.com"

In [3]:
df = pd.read_excel("LP Sched Data.xlsx")

In [4]:
df

,Last Name,First Name,Credentials,Min Shifts,Max Shifts,Min Hours,Max Hours,Min Consecutive AM Shifts,Min Consecutive PM Shifts,Max Consecutive AM Shifts,...,Per Diem,"Block Weekends (Fri, Sat, Sun)",AM Shift Percent,PM Shift Percent,Max PM shifts,Shifts off after PM before next AM,Defiance Percent,Fulton County Percent,Perrysburg Percent,Sylvania Percent
0,Amonette,Melissa,MD,12,15,130.909091,163.636364,0,2,0,...,0,1,0.0,1.0,15,2,0.250000,0.250000,0.250000,0.250000
1,Bahhur,Nael,MD,12,12,130.909091,130.909091,0,2,0,...,0,1,0.0,1.0,12,2,0.000000,1.000000,0.000000,0.000000
2,Baker,Amanda,MD,0,5,0.000000,54.545455,0,0,5,...,1,0,0.0,1.0,10,2,0.000000,1.000000,0.000000,0.000000
3,Baker,Eileen,MD,15,16,163.636364,174.545455,1,2,4,...,0,1,0.8,0.2,3,2,0.333333,0.000000,0.333333,0.333333
4,Barnum,Monica,CNP,0,5,0.000000,54.545455,0,0,5,...,1,0,1.0,0.0,0,2,0.000000,0.333333,0.333333,0.333333
5,Boggs,Thomas,MD,0,5,0.000000,54.545455,0,0,5,...,1,0,0.5,0.5,10,2,0.000000,1.000000,0.000000,0.000000
6,Bretyl,Jen,CNP,0,5,0.000000,54.545455,0,0,5,...,1,0,1.0,0.0,0,2,0.000000,0.333333,0.333333,0.333333
7,Bruss,Mitch,MD,0,5,0.000000,54.545455,0,0,5,...,1,0,0.5,0.5,10,2,0.250000,0.250000,0.250000,0.250000
8,Chavalia,Shannon,CNP,13,13,141.818182,141.818182,1,0,4,...,0,1,1.0,0.0,0,2,0.000000,0.050000,0.050000,0.900000
9,Erwin,Courtney,PA,0,5,0.000000,54.545455,0,0,5,...,1,0,1.0,0.0,0,2,0.000000,0.333300,0.333333,0.333330


In [31]:
# Location
location = ['L' + str(i) for i in range(1, 4)]

# Shifts
shifts = ["Day", "Night", "Day & Night"]
# Define days (1 week)
week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
days = ['Day_' + str(i) + "_" + str(week[i%len(week)]) for i in range(1, 3)]
# Enter shifts of each day
monthly_shifts = {day: shifts for day in days}  # dict with day as key and list of its shifts as value

# Employee credentials
employee_credentials = ["CNP", "DO", "MD", "PA"]
# Enter workers ids (name, number, ...)
# workers = [ ('W' + str(i), employee_credentials[i%len(employee_credentials)] ) for i in range(1, 4)]  # 10 workers available

In [32]:
df["Full Name"] = df["Last Name"] + " " + df["First Name"]

workers = list(df["Full Name"])
workers

['Amonette Melissa',
 'Bahhur Nael',
 'Baker Amanda',
 'Baker Eileen',
 'Barnum Monica',
 'Boggs Thomas',
 'Bretyl Jen',
 'Bruss Mitch',
 'Chavalia Shannon',
 'Erwin Courtney',
 'Goliver Chris',
 'Graber Sara',
 'Gruenbaum Josh',
 'Gruenbaum Marissa',
 'Harwani Ashok',
 'Helfman Todd',
 'Hussain Zak',
 'Meridieth Tom',
 'Miksanek Bryan',
 'Morgan Judy',
 'Murray Adam',
 'Murray Alexandra',
 'Nelson Maureen',
 'Neumeyer Daniel',
 'Payne Nicole',
 'Pontasch Marty',
 'Rashleigh Megan',
 'Romp Eric',
 'Rupp Morgan',
 'Scott Tim',
 'Secory Brandy',
 'Taylor Jay',
 'Tull-Gratop Clara',
 'Worthington Rich',
 'Young Jim',
 'Zeller Kathy']

In [33]:
monthly_shifts

{'Day_1_Tue': ['Day', 'Night', 'Day & Night'],
 'Day_2_Wed': ['Day', 'Night', 'Day & Night']}

In [34]:
workers

['Amonette Melissa',
 'Bahhur Nael',
 'Baker Amanda',
 'Baker Eileen',
 'Barnum Monica',
 'Boggs Thomas',
 'Bretyl Jen',
 'Bruss Mitch',
 'Chavalia Shannon',
 'Erwin Courtney',
 'Goliver Chris',
 'Graber Sara',
 'Gruenbaum Josh',
 'Gruenbaum Marissa',
 'Harwani Ashok',
 'Helfman Todd',
 'Hussain Zak',
 'Meridieth Tom',
 'Miksanek Bryan',
 'Morgan Judy',
 'Murray Adam',
 'Murray Alexandra',
 'Nelson Maureen',
 'Neumeyer Daniel',
 'Payne Nicole',
 'Pontasch Marty',
 'Rashleigh Megan',
 'Romp Eric',
 'Rupp Morgan',
 'Scott Tim',
 'Secory Brandy',
 'Taylor Jay',
 'Tull-Gratop Clara',
 'Worthington Rich',
 'Young Jim',
 'Zeller Kathy']

In [39]:
# df["workers with creds"] = (df["Last Name"] + " " + df["First Name"], df["Credentials"])
# df.to_json(orient="records")
# workers_creds = list(df["workers with creds"])
# workers_creds

workers_creds = []
for record in df.to_dict(orient="records"):
    workers_creds.append((record["Full Name"], record["Credentials"]))

workers_creds

[('Amonette Melissa', 'MD'),
 ('Bahhur Nael', 'MD'),
 ('Baker Amanda', 'MD'),
 ('Baker Eileen', 'MD'),
 ('Barnum Monica', 'CNP'),
 ('Boggs Thomas', 'MD'),
 ('Bretyl Jen', 'CNP'),
 ('Bruss Mitch', 'MD'),
 ('Chavalia Shannon', 'CNP'),
 ('Erwin Courtney', 'PA'),
 ('Goliver Chris', 'MD'),
 ('Graber Sara', 'MD'),
 ('Gruenbaum Josh', 'MD'),
 ('Gruenbaum Marissa', 'PA'),
 ('Harwani Ashok', 'MD'),
 ('Helfman Todd', 'DO'),
 ('Hussain Zak', 'MD'),
 ('Meridieth Tom', 'CNP'),
 ('Miksanek Bryan', 'MD'),
 ('Morgan Judy', 'CNP'),
 ('Murray Adam', 'DO'),
 ('Murray Alexandra', 'DO'),
 ('Nelson Maureen', 'MD'),
 ('Neumeyer Daniel', 'MD'),
 ('Payne Nicole', 'DO'),
 ('Pontasch Marty', 'MD'),
 ('Rashleigh Megan', 'CNP'),
 ('Romp Eric', 'PA'),
 ('Rupp Morgan', 'CNP'),
 ('Scott Tim', 'MD'),
 ('Secory Brandy', 'CNP'),
 ('Taylor Jay', 'MD'),
 ('Tull-Gratop Clara', 'CNP'),
 ('Worthington Rich', 'MD'),
 ('Young Jim', 'MD'),
 ('Zeller Kathy', 'CNP')]

In [40]:
# Initialize model
model = ConcreteModel()

# binary variables representing if a worker is scheduled somewhere
model.works = Var(((worker, cred, day, shift) for worker, cred in workers_creds for day in days for shift in monthly_shifts[day]),
                  within=Binary, initialize=0)

# binary variables representing if a worker is necessary
model.needed = Var(workers, within=Binary, initialize=0)

# binary variables representing if a worker worked on sunday but not on saturday (avoid if possible)
model.no_pref = Var(workers, within=Binary, initialize=0)

In [19]:
# Define an objective function with model as input, to pass later
def obj_rule(m):
    c = len(workers)
    return sum(m.no_pref[worker] for worker in workers) + sum(c * m.needed[worker] for worker in workers)

In [20]:
# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_rule, sense=minimize)

In [44]:
model.works.__dict__

{'_rule_init': <pyomo.core.base.initializer.ConstantInitializer at 0x1fae1b9ef20>,
 '_rule_domain': <pyomo.core.base.set.SetInitializer at 0x1fae1b9e500>,
 '_dense': True,
 '_units': None,
 '_ctype': pyomo.core.base.var.Var,
 'doc': None,
 '_name': 'works',
 '_constructed': True,
 '_parent': <weakref at 0x000001FAE1B6D620; to 'ConcreteModel' at 0x000001FAE1B6D440>,
 '_data': {('Amonette Melissa',
   'MD',
   'Day_1_Tue',
   'Day'): <pyomo.core.base.var._GeneralVarData at 0x1fae12845f0>,
  ('Amonette Melissa',
   'MD',
   'Day_1_Tue',
   'Night'): <pyomo.core.base.var._GeneralVarData at 0x1fae1284660>,
  ('Amonette Melissa',
   'MD',
   'Day_1_Tue',
   'Day & Night'): <pyomo.core.base.var._GeneralVarData at 0x1fae12846d0>,
  ('Amonette Melissa',
   'MD',
   'Day_2_Wed',
   'Day'): <pyomo.core.base.var._GeneralVarData at 0x1fae1284740>,
  ('Amonette Melissa',
   'MD',
   'Day_2_Wed',
   'Night'): <pyomo.core.base.var._GeneralVarData at 0x1fae12847b0>,
  ('Amonette Melissa',
   'MD',
   '

# Constraints

In [21]:
model.constraints = ConstraintList()  # Create a set of constraints

In [42]:
for record in df.to_dict(orient="records"):
    print(record)
    print()
    model.constraints.add(
        40 >= sum(8 * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )

{'Last Name': 'Amonette', 'First Name': 'Melissa', 'Credentials': 'MD', 'Min Shifts': 12, 'Max Shifts': 15, 'Min Hours': 130.9090909090909, 'Max Hours': 163.63636363636363, 'Min Consecutive AM Shifts': 0, 'Min Consecutive PM Shifts': 2, 'Max Consecutive AM Shifts': 0, 'Max Consecutive PM Shifts': 5, 'Full Time': 1, 'Part Time': 0, 'Per Diem': 0, 'Block Weekends (Fri, Sat, Sun)': 1, 'AM Shift Percent': 0.0, 'PM Shift Percent': 1.0, 'Max PM shifts': 15, 'Shifts off after PM before next AM': 2, 'Defiance Percent': 0.25, 'Fulton County Percent': 0.25, 'Perrysburg Percent': 0.25, 'Sylvania Percent': 0.25, 'Full Name': 'Amonette Melissa'}

{'Last Name': 'Bahhur', 'First Name': 'Nael', 'Credentials': 'MD', 'Min Shifts': 12, 'Max Shifts': 12, 'Min Hours': 130.9090909090909, 'Max Hours': 130.9090909090909, 'Min Consecutive AM Shifts': 0, 'Min Consecutive PM Shifts': 2, 'Max Consecutive AM Shifts': 0, 'Max Consecutive PM Shifts': 5, 'Full Time': 1, 'Part Time': 0, 'Per Diem': 0, 'Block Weekends 

In [ ]:
# Constraint: no more than 40 hours worked
for worker in workers:
    model.constraints.add(
        40 >= sum(8 * model.works[worker, day, shift] for day in days for shift in days_shifts[day])
    )